# Experiment 10 - Implementing a Bidirectional Encoder Representation with Transformers (BERT) for 

## About the BERT
BERT stands for "Bidirectional Encoder Representation with Transformers". To put it in simple words BERT extracts patterns or representations from the data or word embeddings by passing it through an encoder. The encoder itself is a transformer architecture that is stacked together. It is a bidirectional transformer which means that during training it considers the context from both left and right of the vocabulary to extract patterns or representations.

![](./assets/BERT-encoder.webp)

## Paradigms of Use
BERT uses two training paradigms: Pre-training and Fine-tuning. 

During pre-training, the model is trained on a large dataset to extract patterns. This is generally an unsupervised learning task where the model is trained on an unlabelled dataset like the data from a big corpus like Wikipedia.  

During fine-tuning the model is trained for downstream tasks like Classification, Text-Generation, Language Translation, Question-Answering, and so forth. Essentially, you can download a pre-trained model and then Transfer-learn the model on your data.

## How does BERT work?
BERT makes use of Transformer, an attention mechanism that learns contextual relations between words (or sub-words) in a text. In its vanilla form, Transformer includes two separate mechanisms — an encoder that reads the text input and a decoder that produces a prediction for the task. Since BERT’s goal is to generate a language model, only the encoder mechanism is necessary. The detailed workings of Transformer are described in a paper by Google.

As opposed to directional models, which read the text input sequentially (left-to-right or right-to-left), the Transformer encoder reads the entire sequence of words at once. Therefore it is considered bidirectional, though it would be more accurate to say that it’s non-directional. This characteristic allows the model to learn the context of a word based on all of its surroundings (left and right of the word).

When training language models, there is a challenge of defining a prediction goal. Many models predict the next word in a sequence (e.g. “The child came home from ___”), a directional approach which inherently limits context learning. To overcome this challenge, BERT uses two training strategies:

### Masked LM (MLM)

Before feeding word sequences into BERT, 15% of the words in each sequence are replaced with a [MASK] token. The model then attempts to predict the original value of the masked words, based on the context provided by the other, non-masked, words in the sequence. In technical terms, the prediction of the output words requires:
1. Adding a classification layer on top of the encoder output.
2. Multiplying the output vectors by the embedding matrix, transforming them into the vocabulary dimension.
3. Calculating the probability of each word in the vocabulary with softmax.

![](./assets/BERT-architecture.webp)

The BERT loss function takes into consideration only the prediction of the masked values and ignores the prediction of the non-masked words. As a consequence, the model converges slower than directional models, a characteristic which is offset by its increased context awareness.

### Next Sentence Prediction (NSP)

In the BERT training process, the model receives pairs of sentences as input and learns to predict if the second sentence in the pair is the subsequent sentence in the original document. During training, 50% of the inputs are a pair in which the second sentence is the subsequent sentence in the original document, while in the other 50% a random sentence from the corpus is chosen as the second sentence. The assumption is that the random sentence will be disconnected from the first sentence.

To help the model distinguish between the two sentences in training, the input is processed in the following way before entering the model:

1. A [CLS] token is inserted at the beginning of the first sentence and a [SEP] token is inserted at the end of each sentence.
2. A sentence embedding indicating Sentence A or Sentence B is added to each token. Sentence embeddings are similar in concept to token embeddings with a vocabulary of 2.
3. A positional embedding is added to each token to indicate its position in the sequence. The concept and implementation of positional embedding are presented in the Transformer paper.

![](./assets/BERT-with-modifications.webp)

Source: BERT [Devlin et al., 2018], with modifications

To predict if the second sentence is indeed connected to the first, the following steps are performed:

1. The entire input sequence goes through the Transformer model.
2. The output of the [CLS] token is transformed into a 2×1 shaped vector, using a simple classification layer (learned matrices of weights and biases).
3. Calculating the probability of IsNextSequence with softmax.

When training the BERT model, Masked LM and Next Sentence Prediction are trained together, with the goal of minimizing the combined loss function of the two strategies.

## How to use BERT (Fine-tuning)

Using BERT for a specific task is relatively straightforward:

BERT can be used for a wide variety of language tasks, while only adding a small layer to the core model:

1. Classification tasks such as sentiment analysis are done similarly to Next Sentence classification, by adding a classification layer on top of the Transformer output for the [CLS] token.
2. In Question Answering tasks (e.g. SQuAD v1.1), the software receives a question regarding a text sequence and is required to mark the answer in the sequence. Using BERT, a Q&A model can be trained by learning two extra vectors that mark the beginning and the end of the answer.
3. In Named Entity Recognition (NER), the software receives a text sequence and is required to mark the various types of entities (Person, Organization, Date, etc) that appear in the text. Using BERT, a NER model can be trained by feeding the output vector of each token into a classification layer that predicts the NER label.

In the fine-tuning training, most hyper-parameters stay the same as in BERT training.

## Summary

BERT falls into a self-supervised model. That means, it can generate inputs and labels from the raw corpus without being explicitly programmed by humans. Remember the data it is trained on is unstructured.

BERT was pre-trained with two specific tasks: Masked Language Model and Next sentence prediction. The former uses masked input like “the man [MASK] to the store” instead of “the man went to the store”. This restricts BERT to see the words next to it which allows it to learn bidirectional representations as much as possible making it much more flexible and reliable for several downstream tasks. The latter predicts whether the two sentences are contextually assigned to each other. 

![](./assets/BERT-overview.webp)

## Code

### Importing Libraries

In [3]:
import torch
import re

### Preprocessing

#### Creating our corpus of raw text

In [2]:
text = (
    'Hello, how are you? I am Romeo.\n'
    'Hello, Romeo My name is Juliet. Nice to meet you.\n'
    'Nice meet you too. How are you today?\n'
    'Great. My baseball team won the competition.\n'
    'Oh Congratulations, Juliet\n'
    'Thanks you Romeo'
)

#### Cleaning our raw textual data

Filtering out all of these special characters: `filter: ['.', ',', '?', '!']`

In [9]:
listSentences = re.sub("[.,!?-]", '', text.lower()).split('\n')
  
listWords = list(set(" ".join(listSentences).split()))

Printing out the split list of sentences.

In [10]:
listSentences

['hello how are you i am romeo',
 'hello romeo my name is juliet nice to meet you',
 'nice meet you too how are you today',
 'great my baseball team won the competition',
 'oh congratulations juliet',
 'thanks you romeo']

Same for words.

In [11]:
listWords

['won',
 'i',
 'am',
 'juliet',
 'name',
 'competition',
 'congratulations',
 'to',
 'how',
 'romeo',
 'today',
 'thanks',
 'my',
 'nice',
 'oh',
 'you',
 'great',
 'is',
 'are',
 'hello',
 'the',
 'meet',
 'too',
 'team',
 'baseball']

#### Creating special tags for the tokens pre-defined by BERT

BERT takes special tokens during training. Here is a table explaining the purpose of various tokens:
|                                            Token                                            |                        Purpose                       |
|:-------------------------------------------------------------------------------------------:|:----------------------------------------------------:|
| [CLS]                                                                                       |       The first token is always classification       |
| [SEP]                                                                                       |                Separates two sentences               |
| [END]                                                                                       |                   End the sentence.                  |
| [PAD]                                                                                       |    Use to truncate the sentence with equal length.   |
|                                            [MASK]                                           | Use to create a mask by replacing the original word. |

#### Assigning indexes by count with reference to our cleaned corpus

In [13]:
dictionaryWords = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}

for idx, word in enumerate(listWords):
   dictionaryWords[word] = idx + 4
   serialise = {idx: word for idx, word in enumerate(dictionaryWords)}
   vocabularySize = len(dictionaryWords)

In [15]:
vocabularySize

29

In [16]:
serialise

{0: '[PAD]',
 1: '[CLS]',
 2: '[SEP]',
 3: '[MASK]',
 4: 'won',
 5: 'i',
 6: 'am',
 7: 'juliet',
 8: 'name',
 9: 'competition',
 10: 'congratulations',
 11: 'to',
 12: 'how',
 13: 'romeo',
 14: 'today',
 15: 'thanks',
 16: 'my',
 17: 'nice',
 18: 'oh',
 19: 'you',
 20: 'great',
 21: 'is',
 22: 'are',
 23: 'hello',
 24: 'the',
 25: 'meet',
 26: 'too',
 27: 'team',
 28: 'baseball'}

In [14]:
dictionaryWords

{'[PAD]': 0,
 '[CLS]': 1,
 '[SEP]': 2,
 '[MASK]': 3,
 'won': 4,
 'i': 5,
 'am': 6,
 'juliet': 7,
 'name': 8,
 'competition': 9,
 'congratulations': 10,
 'to': 11,
 'how': 12,
 'romeo': 13,
 'today': 14,
 'thanks': 15,
 'my': 16,
 'nice': 17,
 'oh': 18,
 'you': 19,
 'great': 20,
 'is': 21,
 'are': 22,
 'hello': 23,
 'the': 24,
 'meet': 25,
 'too': 26,
 'team': 27,
 'baseball': 28}

### Masking

In [ ]:
def make_batch(batchSize):
    batch = []
    positive = negative = 0
    while positive != batchSize / 2 or negative != batchSize / 2:
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))

        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]

        input_ids = [dictionaryWords['[CLS]']] + tokens_a + [dictionaryWords['[SEP]']] + tokens_b + [dictionaryWords['[SEP]']]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        # MASK LM
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # 15 % of tokens in one sentence
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                            if token != dictionaryWords['[CLS]'] and token != dictionaryWords['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.8:  # 80%
                input_ids[pos] = dictionaryWords['[MASK]'] # make mask
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = dictionaryWords[number_dict[index]] # replace

        # Zero Paddings
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batchSize/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batchSize/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
    return batch